# The Battle of Neighborhoods

## Introduction/Business Problem

### Background
There are 58 neighborhoods in Bloomington, IL based on the information from localWiki. There are State Farm headquarter, Pepsi Ice Center, Grossinger Motors Arena in the city. Different neighborhoods have different majority venues. Visitors coming to the city usually have limited time. How to recommend the visitors to the right neighborhood is important to the development of the city's tourism.

### Description of the problem
If a visitor comes to Bloominton and he likes outdoor activities and Chinese food, which neighborhood he should go. So he can play and eat well. The solution of the problem can be utilized in Bloominton's city website as a guidance to the city.

## Data
### Neighborhoods data scraped from Next Door website
Initial bloomington_neighborhoods data set scraped from Next Door has 90 rows(Neighborhood) and 3 varaibles(neighborhood name, latitude, longitude)

In [179]:
# import libraries to get data
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [180]:
# get neighborhoods
website_url = requests.get('https://nextdoor.com/city/bloomington--il/').text
soup = BeautifulSoup(website_url,'lxml')
bloomington_neighborhoods = {'neighborhoods': [], 'latitude': [], 'longitude': []}
soup_neighborhoods = soup.select('div.hood_group p a')
for i in range(len(soup_neighborhoods)):
    bloomington_neighborhoods['neighborhoods'].append(soup_neighborhoods[i].get_text())
    bloomington_neighborhoods['latitude'].append(np.nan)
    bloomington_neighborhoods['longitude'].append(np.nan)
bloomington_neighborhoods = pd.DataFrame(bloomington_neighborhoods)

In [181]:
bloomington_neighborhoods.shape

(90, 3)

### Latitude and longitude from geopy package in Python
Use the neighborhood name to get the coordinate information, there are 23 listed neighborhoods whose coordinate information is not available. So the data set has 67 rows(neighborhoods), 3 variables(neighborhood name, latitude, longitude)

After checking the summary statistics of latitude and longitude, there are a few points which are potentially incorrect. After removing the incorrect points. The final data set has 59 rows, 3 variables

In [182]:
# get the latitude and longitude of Bloomington
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
address = 'Bloomington, IL'
geolocator = Nominatim(user_agent="bl_explorer", timeout=7)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bloomington are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bloomington are 40.4731073, -88.9941403.


In [183]:
# get the latitude and longitude of neighborhoods in Bloomington
for neighborhood in bloomington_neighborhoods['neighborhoods']:
    full_address = neighborhood + ', Bloomington, IL'
    location = geolocator.geocode(full_address)
    if location:
        latitude = location.latitude
        longitude = location.longitude
        bloomington_neighborhoods.loc[bloomington_neighborhoods['neighborhoods'] == neighborhood, 'latitude'] = latitude
        bloomington_neighborhoods.loc[bloomington_neighborhoods['neighborhoods'] == neighborhood, 'longitude'] = longitude
    else:
        print("{} doesn't have coordinate information".format(full_address))

Alexander Estates, Bloomington, IL doesn't have coordinate information
Cardinal Ridge/South gate, Bloomington, IL doesn't have coordinate information
Cherrywood Hilltop, Bloomington, IL doesn't have coordinate information
Country Acres, Bloomington, IL doesn't have coordinate information
Country Club Place-Northcrest, Bloomington, IL doesn't have coordinate information
Crestwicke, Bloomington, IL doesn't have coordinate information
Dimmitt's Grove, Bloomington, IL doesn't have coordinate information
Euclid/Oakland, Bloomington, IL doesn't have coordinate information
Evergreen-Winchester, Bloomington, IL doesn't have coordinate information
Fairway Knolls, Bloomington, IL doesn't have coordinate information
Grove at Kickapoo Creek, Bloomington, IL doesn't have coordinate information
Hillcrest/Clearwater Park, Bloomington, IL doesn't have coordinate information
Hilltop Mobile  Park, Bloomington, IL doesn't have coordinate information
Lincolnwood & Holiday Knolls, Bloomington, IL doesn't h

In [184]:
# Fill the missing values by searching in google
# Alexander Estates
bloomington_neighborhoods.iloc[0, 1] = 40.4646
bloomington_neighborhoods.iloc[0, 2] = -89.0214
# Cardinal Ridge/South gate, no direct result for this

# Cherrywood Hilltop, no direct result

# Country Acres, no direct result

# Country Club Place-Northcrest, no direct result

# Crestwicke
bloomington_neighborhoods.iloc[13, 1] = 40.4158
bloomington_neighborhoods.iloc[13, 2] = -88.9659

# Dimmitt's Grove

# Euclid/Oakland

# Evergreen-Winchester

# Fairway Knolls
bloomington_neighborhoods.iloc[29, 1] = 40.4935
bloomington_neighborhoods.iloc[29, 2] = -88.9539

# Grove at Kickapoo Creek

# Hillcrest/Clearwater Park

# Hilltop Mobile Park

# Lincolnwood & Holiday Knolls

# Near East Side

# Northern Wingate Rd

# OFLGA

# Old East Side

# Pinebach
bloomington_neighborhoods.iloc[61, 1] = 40.4922
bloomington_neighborhoods.iloc[61, 2] = -88.9707

# SoDimm

# South Mercer Corridor

# SWBRA

# The Davis Jefferson Neighborhood Association

# Trails on Sunset Lake

# White Oak and Hovey

# White's Place

# Wood Hill Towers

# drop neighborhood still has na values, 67 neighborhoods left
bloomington_neighborhoods = bloomington_neighborhoods.dropna().reset_index(drop=True)


In [185]:
# summary statsitics, min and max value are suspicious
bloomington_neighborhoods[['latitude', 'longitude']].describe()

,latitude,longitude
count,67.000000,67.000000
mean,40.638309,-89.425026
std,1.175825,3.277193
min,37.097551,-113.610761
25%,40.464642,-88.994140
50%,40.483925,-88.936313
75%,40.498886,-88.921122
max,44.950321,-83.737892


In [186]:
first4_index = bloomington_neighborhoods['latitude'].sort_values(ascending = True).index[0:4]
last4_index = bloomington_neighborhoods['latitude'].sort_values(ascending = True).index[-4:]
eight_index = first4_index.union(last4_index)
wrong_points = bloomington_neighborhoods.iloc[eight_index]

In [187]:
wrong_points

,neighborhoods,latitude,longitude
5,Brookstone,44.853688,-93.240502
10,Deer Ridge,44.843261,-93.375264
24,Founders Grove,44.796006,-93.370440
37,Indian Hills,37.097551,-113.610761
38,Lakeside,39.070870,-86.376385
40,Midtown,44.950321,-93.252494
47,Prairie Place,39.507354,-83.737892
54,Sheridan,39.155867,-86.522539


In [188]:
# create map for wrong points
map_wrong_points = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lon, poi in zip(wrong_points['latitude'], wrong_points['longitude'], wrong_points['neighborhoods']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_wrong_points)
map_wrong_points

In [189]:
# turns out the four points are out of bloominton, so remove it 
bloomington_neighborhoods = bloomington_neighborhoods.drop(labels = eight_index, axis= 0).reset_index(drop = True)

In [190]:
bloomington_neighborhoods[['latitude', 'longitude']].describe()

,latitude,longitude
count,59.000000,59.000000
mean,40.482911,-88.948991
std,0.019938,0.037098
min,40.415800,-89.021400
25%,40.467756,-88.985678
50%,40.483925,-88.935246
75%,40.497160,-88.921713
max,40.515680,-88.893739


In [191]:
bloomington_neighborhoods.shape

(59, 3)

In [192]:
# create map
map_bloominton = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lon, poi in zip(bloomington_neighborhoods['latitude'], bloomington_neighborhoods['longitude'], bloomington_neighborhoods['neighborhoods']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_bloominton)
map_bloominton

### Venues data from 4square API
Use neighborhood coordinates information and get venues data from 4square API. The neighborhood_venues data set has 345 rows(venues) and 7 variables(neighborhood name, neighborhood latitude, neighborhood longitude, venue name, venue latitude, venue longitude, venue category)

In [193]:
# Define foursquare credentials and version
CLIENT_ID = 'QQ5UMYZIDNTTVGMTJJUMO2Z4RIAVMAAFOXG55FQYYWVIADIB' # your Foursquare ID
CLIENT_SECRET = 'X2MYQDLFZYYQMROJB1JR3RX10YLOJBXNGUXF2NJE5DBEJHF5' # your Foursquare Secret
VERSION = '20191215' # Foursquare API version
LIMIT = 100
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QQ5UMYZIDNTTVGMTJJUMO2Z4RIAVMAAFOXG55FQYYWVIADIB
CLIENT_SECRET:X2MYQDLFZYYQMROJB1JR3RX10YLOJBXNGUXF2NJE5DBEJHF5


In [194]:
# get venues from one neighborhood
neighborhood_latitude = bloomington_neighborhoods.iloc[0, 1]
neighborhood_longitude = bloomington_neighborhoods.iloc[0, 2]
neighborhood_name = bloomington_neighborhoods.iloc[0, 0]
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
venues = requests.get(url).json()
# print(json.dumps(venues, indent = 4))

Latitude and longitude values of Alexander Estates are 40.4646, -89.0214.


In [195]:
# devise function for getting venues for all neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    venues_list=[] # each element represents a list of venues in a neighborhood
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
#         if name == 'Wilder Dr':
#             print(json.dumps(requests.get(url).json(), indent = 4))
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [196]:
# get venues for all neighborhoods
neighborhoods_venues = getNearbyVenues(bloomington_neighborhoods['neighborhoods'], 
                                      bloomington_neighborhoods['latitude'],
                                      bloomington_neighborhoods['longitude'])

In [213]:
neighborhoods_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alexander Estates,40.46460,-89.021400,Gold's Gym,40.463958,-89.016088,Gym
1,Alexander Estates,40.46460,-89.021400,In Maple Grove,40.465580,-89.018420,Outdoors & Recreation
2,Alexander Estates,40.46460,-89.021400,Owen's\t Nursery,40.465250,-89.025408,Garden Center
3,Bloomington Heights,40.48245,-89.019091,Chase Bank,40.483826,-89.018199,Bank
4,Bloomington Heights,40.48245,-89.019091,Popeyes Louisiana Kitchen,40.484142,-89.020360,Fried Chicken Joint


In [198]:
print('There are {} uniques categories.'.format(len(neighborhoods_venues['Venue Category'].unique())))

There are 110 uniques categories.


One hot encoding the venue category, add neighborhood column and group by neighborhood, calculate the mean frequency of each category. The neighborhoods_venues_category_grouped has 57 rows(neighborhood), means there are two neighborhoods which don't return any venues. It has 111 columns, first column represent the neighborhood name, the rest are each category of the venues.

In [199]:
# one-hot encoding the venue category
neighborhoods_venues_category = pd.get_dummies(neighborhoods_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column
neighborhoods_venues_category.insert(loc=0, column='Neighborhood', value=neighborhoods_venues['Neighborhood'])
# group by neighborhood, some neighborhoods don't have venues
neighborhoods_venues_category_grouped = neighborhoods_venues_category.groupby(by = 'Neighborhood').mean().reset_index()
sorted_num_venues_neighborhood = neighborhoods_venues_category.groupby(by = 'Neighborhood').count().reset_index().sort_values(by = 'Accessories Store', ascending = False)['Neighborhood']

Generate top 5 venues category for each neighborhood, neighborhoods_venues_category_grouped_sorted has 57 rows(neighborhood), 6 columns(neighborhood name, 5 most common venue categories)

In [200]:
# define a function to return the venues with most frequency
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [201]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_category_grouped_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_category_grouped_sorted['Neighborhood'] = neighborhoods_venues_category_grouped['Neighborhood']
for i in np.arange(neighborhoods_venues_category_grouped_sorted.shape[0]):
    neighborhoods_venues_category_grouped_sorted.iloc[i, 1:] = return_most_common_venues(neighborhoods_venues_category_grouped.iloc[i, 1:], num_top_venues)

## Methodology
### Clustering
Because the neighborhood number is small, so I choose 3 clusters

In [202]:
# clustering on neighborhoods
# set number of clusters
kclusters = 3
neighborhoods_venues_category_grouped_clustering = neighborhoods_venues_category_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhoods_venues_category_grouped_clustering)

# insert cluster information back to sorted neighborhood group
neighborhoods_venues_category_grouped_sorted.insert(loc=0, column='cluster', value=kmeans.labels_)
neighborhoods_venues_category_grouped_sorted = neighborhoods_venues_category_grouped_sorted.rename(columns={'Neighborhood': 'neighborhoods'})
neighborhoods_venues_merged = pd.merge(bloomington_neighborhoods, neighborhoods_venues_category_grouped_sorted, on = 'neighborhoods')

## Results

In [203]:
# visualize again
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_venues_merged['latitude'], neighborhoods_venues_merged['longitude'], neighborhoods_venues_merged['neighborhoods'], neighborhoods_venues_merged['cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [204]:
# cluster 1
neighborhoods_venues_merged.loc[neighborhoods_venues_merged['cluster'] == 0, 
                                neighborhoods_venues_merged.columns[[0] + list(range(4, 9))]].head()

,neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Alexander Estates,Outdoors & Recreation,Gym,Garden Center,Zoo,Gay Bar
1,Bloomington Heights,Hotel,Fast Food Restaurant,Grocery Store,Fish & Chips Shop,Fried Chicken Joint
4,Clobertin Ct,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Video Store,Furniture / Home Store
5,Colton,Pharmacy,Baseball Field,Brewery,Candy Store,Zoo
6,Crestwicke,Golf Course,Gay Bar,Electronics Store,Exhibit,Farmers Market


In [205]:
# cluster 2
neighborhoods_venues_merged.loc[neighborhoods_venues_merged['cluster'] == 1, 
                                neighborhoods_venues_merged.columns[[0] + list(range(4, 9))]].head()

,neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
11,Eagle Crest,Trail,Park,Zoo,Gas Station,Electronics Store
13,Eagle View,Playground,Park,Zoo,Gas Station,Exhibit
19,Fleetwood,Intersection,Theater,Lawyer,Park,Zoo
21,Gaelic Place,Playground,Park,Gym / Fitness Center,Zoo,Gas Station
25,Hawthorne Hills,Park,Zoo,Gay Bar,Exhibit,Farmers Market


In [206]:
# cluster 3
neighborhoods_venues_merged.loc[neighborhoods_venues_merged['cluster'] == 2, 
                                neighborhoods_venues_merged.columns[[0] + list(range(4, 9))]]

,neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Breckenridge Village,American Restaurant,Lake,Golf Course,Exhibit,Farmers Market
3,Breckenwood Ct,American Restaurant,Gay Bar,Electronics Store,Exhibit,Farmers Market
39,Prairie View South,American Restaurant,Gay Bar,Electronics Store,Exhibit,Farmers Market
52,White Eagle South,American Restaurant,Lake,Golf Course,Exhibit,Farmers Market


Chinese token appears first in the 3rd Most Common Venue. Find the neighborhood Clobertin Ct contains chinese restaurant, and more information about this neighborhood, looks like this neighborhood doesn't have lots of outdoor activities

In [207]:
neighborhoods_venues_merged[neighborhoods_venues_merged['3rd Most Common Venue'].str.contains('chinese', case = False)]

,neighborhoods,latitude,longitude,cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Clobertin Ct,40.470827,-88.954543,0,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Video Store,Furniture / Home Store


In [208]:
neighborhoods_venues.loc[neighborhoods_venues['Neighborhood'] == 'Clobertin Ct'].loc[neighborhoods_venues.loc[neighborhoods_venues['Neighborhood'] == 'Clobertin Ct']['Venue Category'].str.contains('chinese', case = False)]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
33,Clobertin Ct,40.470827,-88.954543,China Star,40.473810,-88.956851,Chinese Restaurant
37,Clobertin Ct,40.470827,-88.954543,Lucky Garden,40.472074,-88.951661,Chinese Restaurant


In [209]:
num_venues = neighborhoods_venues.loc[neighborhoods_venues['Neighborhood'] == 'Clobertin Ct'].shape[0]
print('There are {} venues inside the neighborhood'.format(num_venues))

There are 23 venues inside the neighborhood


Find the neighborhood contains places for outdoor activities, there are a couple potential neighborhoods, first rank neighborhoods by the number of venues inside the neighborhood, then based on the rank, find the neighborhood E Grove Street has the 1st Most Common as Park.

In [210]:
neighborhoods_venues_category_grouped_sorted.iloc[sorted_num_venues_neighborhood.index]

,cluster,neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
23,0,Gridley St,Bar,Sandwich Place,Park,Italian Restaurant,Doctor's Office
7,0,Downtown Bloomington,Bar,Sandwich Place,Hockey Arena,Italian Restaurant,Farmers Market
1,0,Bloomington Heights,Hotel,Fast Food Restaurant,Grocery Store,Fish & Chips Shop,Fried Chicken Joint
4,0,Clobertin Ct,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Video Store,Furniture / Home Store
10,0,E Grove St,Park,Convenience Store,Bar,Pharmacy,Salon / Barbershop
20,0,Franklin Park,Bar,Hobby Shop,Brewery,Park,Optical Shop
38,0,Pinebach,American Restaurant,Breakfast Spot,Discount Store,Fast Food Restaurant,Paper / Office Supplies Store
18,0,Fairway Knolls,Furniture / Home Store,Mobile Phone Shop,Gym,Hotel,Pet Store
22,0,Golden Eagle South,Medical Center,Playground,Plaza,Trail,Athletics & Sports
32,0,McGraw Park,Playground,Baseball Field,Trail,Athletics & Sports,Medical Center


In [211]:
neighborhoods_venues.loc[neighborhoods_venues['Neighborhood'] == 'E Grove St']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
114,E Grove St,40.478071,-88.981431,BEER NUTS Brand Snacks,40.479303,-88.982701,Snack Place
115,E Grove St,40.478071,-88.981431,Schooners,40.478105,-88.985332,Bar
116,E Grove St,40.478071,-88.981431,Grove Street Bakery,40.478111,-88.985364,Bakery
117,E Grove St,40.478071,-88.981431,Vrooman Mansion B & B,40.475992,-88.985428,Bed & Breakfast
118,E Grove St,40.478071,-88.981431,Circle K,40.478107,-88.985351,Convenience Store
119,E Grove St,40.478071,-88.981431,David Davis Mansion,40.481672,-88.984302,Historic Site
120,E Grove St,40.478071,-88.981431,Urban Carnivore,40.477667,-88.980006,Pet Store
121,E Grove St,40.478071,-88.981431,Coffee Hound 3,40.479647,-88.981606,Coffee Shop
122,E Grove St,40.478071,-88.981431,Green Top Grocery,40.479650,-88.980513,Grocery Store
123,E Grove St,40.478071,-88.981431,First Edition,40.480067,-88.983166,Salon / Barbershop


In [212]:
num_venues = neighborhoods_venues.loc[neighborhoods_venues['Neighborhood'] == 'E Grove St'].shape[0]
print('There are {} venues inside the neighborhood'.format(num_venues))

There are 17 venues inside the neighborhood


## Discussion

From the clustering result, the first cluster is a group of neighborhood which contains mixed outdoor venues, restaurants, and stores.

The second cluster is a group of neighborhood which contains outdoor venues.

The third cluster is a group of neighborhood which contains restaurants

## Conclusion

If such a visitor comes Bloomington, I will recommend him to go visiting E Grove St neighborhood. 

For lunch or dinner, I will recommend him to go visiting Clobertin Ct neighborhood